In [98]:
import pandas as pd
from psql_config import config
import psycopg2

params=config()
conn=psycopg2.connect(**params)
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

cur = conn.cursor()
sales_data = create_pandas_table("SELECT week_of_year, sales, total_guest_count, fiscal_year, location FROM sales WHERE fiscal_year > 2017 ORDER BY week_of_year, fiscal_year")
cur.close()
conn.close()

df=sales_data

df.fillna(0,inplace=True)

#find current year
current_year=df['fiscal_year'].max()

#find current week based on last entry in df
curr_week=df[df['fiscal_year']==current_year]['week_of_year'].max()

df=df[(df['week_of_year'] >= curr_week-6) & (df['week_of_year'] <= curr_week+6)]

df_agg_sales=df.groupby(['fiscal_year','week_of_year'])['sales'].sum().reset_index()
df_agg_gc=df.groupby(['fiscal_year','week_of_year'])['total_guest_count'].sum().reset_index()

df_agg_sales.sort_values(by=['week_of_year','fiscal_year'],inplace=True)
df_agg_gc.sort_values(by=['week_of_year','fiscal_year'],inplace=True)

df_agg_sales['percent_sales']=df_agg_sales['sales'].pct_change().round(4)*100
df_agg_gc['percent_guest_count']=df_agg_gc['total_guest_count'].pct_change().round(4)*100
df_agg_sales.sort_values(by=['fiscal_year','week_of_year'],inplace=True)
df_agg_gc.sort_values(by=['fiscal_year','week_of_year'],inplace=True)

In [99]:
df_agg_gc

,fiscal_year,week_of_year,total_guest_count,percent_guest_count
0,2018,10,59283.0,NaN
1,2018,11,63657.0,-43.75
2,2018,12,60238.0,-39.77
3,2018,13,62201.0,-19.57
4,2018,14,58772.0,-20.85
5,2018,15,59738.0,-6.94
6,2018,16,63355.0,-18.88
7,2018,17,62603.0,-19.22
8,2018,18,61171.0,-5.44
9,2018,19,64492.0,0.44


In [100]:
df_total=df_agg_sales.join(df_agg_gc, lsuffix='_l',rsuffix='_r')
df_total

,fiscal_year_l,week_of_year_l,sales,percent_sales,fiscal_year_r,week_of_year_r,total_guest_count,percent_guest_count
0,2018,10,1018265.0,NaN,2018,10,59283.0,NaN
1,2018,11,1042652.0,-4.63,2018,11,63657.0,-43.75
2,2018,12,1028983.0,6.43,2018,12,60238.0,-39.77
3,2018,13,1125776.0,54.64,2018,13,62201.0,-19.57
4,2018,14,999711.0,50.01,2018,14,58772.0,-20.85
5,2018,15,1028957.0,57.08,2018,15,59738.0,-6.94
6,2018,16,1163814.0,40.91,2018,16,63355.0,-18.88
7,2018,17,1190015.0,49.77,2018,17,62603.0,-19.22
8,2018,18,1104017.8,-1.53,2018,18,61171.0,-5.44
9,2018,19,1253423.0,11.64,2018,19,64492.0,0.44


In [101]:
df_total.drop(columns=['fiscal_year_r','week_of_year_r'],inplace=True)
df_total.rename(columns={'fiscal_year_l':'fiscal_year','week_of_year_l':'week_of_year'},inplace=True)

In [102]:
df_total

,fiscal_year,week_of_year,sales,percent_sales,total_guest_count,percent_guest_count
0,2018,10,1018265.0,NaN,59283.0,NaN
1,2018,11,1042652.0,-4.63,63657.0,-43.75
2,2018,12,1028983.0,6.43,60238.0,-39.77
3,2018,13,1125776.0,54.64,62201.0,-19.57
4,2018,14,999711.0,50.01,58772.0,-20.85
5,2018,15,1028957.0,57.08,59738.0,-6.94
6,2018,16,1163814.0,40.91,63355.0,-18.88
7,2018,17,1190015.0,49.77,62603.0,-19.22
8,2018,18,1104017.8,-1.53,61171.0,-5.44
9,2018,19,1253423.0,11.64,64492.0,0.44
